This tutorial and the assets can be downloaded as part of the [Wallaroo Tutorials repository](https://github.com/WallarooLabs/Wallaroo_Tutorials/tree/main/wallaroo-testing-tutorials/abtesting).

## A/B Testing

A/B testing is a method that provides the ability to test out ML models for performance, accuracy or other useful benchmarks.  A/B testing is contrasted with the Wallaroo Shadow Deployment feature.  In both cases, two sets of models are added to a pipeline step:

* Control or Champion model:  The model currently used for inferences.
* Challenger model(s): One or more models that are to be compared to the champion model.

The two feature are different in this way:

| Feature | Description |
|---|---|
| A/B Testing | A subset of inferences are submitted to either the champion ML model or a challenger ML model. |
| Shadow Deploy | All inferences are submitted to the champion model and one or more challenger models. |

So to repeat:  A/B testing submits *some* of the inference requests to the champion model, some to the challenger model with one set of outputs, while shadow testing submits *all* of the inference requests to champion and shadow models, and has separate outputs.

This tutorial demonstrate how to conduct A/B testing in Wallaroo.  For this example we will be using an open source model that uses an [Aloha CNN LSTM model](https://www.researchgate.net/publication/348920204_Using_Auxiliary_Inputs_in_Deep_Learning_Models_for_Detecting_DGA-based_Domain_Names) for classifying Domain names as being either legitimate or being used for nefarious purposes such as malware distribution.  

For our example, we will perform the following:

* Create a workspace for our work.
* Upload the Aloha model and a challenger model.
* Create a pipeline that can ingest our submitted data with the champion model and the challenger model set into a A/B step.
* Run a series of sample inferences to display inferences that are run through the champion model versus the challenger model, then determine which is more efficient.

All sample data and models are available through the [Wallaroo Quick Start Guide Samples repository](https://github.com/WallarooLabs/quickstartguide_samples).

## Steps

### Import libraries

Here we will import the libraries needed for this notebook.

In [35]:
import wallaroo
from wallaroo.object import EntityNotFoundError
import os
import pandas as pd
import json
from IPython.display import display

# used to display dataframe information without truncating
from IPython.display import display
pd.set_option('display.max_colwidth', None)

In [36]:
wallaroo.__version__

'2023.1.0'

### Arrow Support

As of the 2023.1 release, Wallaroo provides support for dataframe and Arrow for inference inputs.  This tutorial allows users to adjust their experience based on whether they have enabled Arrow support in their Wallaroo instance or not.

If Arrow support has been enabled, `arrowEnabled=True`. If disabled or you're not sure, set it to `arrowEnabled=False`

The examples below will be shown in an arrow enabled environment.

In [37]:
import os
# Only set the below to make the OS environment ARROW_ENABLED to TRUE.  Otherwise, leave as is.
os.environ["ARROW_ENABLED"]="True"

if "ARROW_ENABLED" not in os.environ or os.environ["ARROW_ENABLED"].casefold() == "False".casefold():
    arrowEnabled = False
else:
    arrowEnabled = True
print(arrowEnabled)

True


### Connect to the Wallaroo Instance

This command will be used to set up a connection to the Wallaroo cluster and allow creating and use of Wallaroo inference engines.

In [38]:
# Client connection from local Wallaroo instance

# wl = wallaroo.Client()

# SSO login through keycloak

wallarooPrefix = "YOUR PREFIX"
wallarooSuffix = "YOUR SUFFIX"

wallarooPrefix = "sparkly-apple-3026"
wallarooSuffix = "wallaroo.community"

wl = wallaroo.Client(api_endpoint=f"https://{wallarooPrefix}.api.{wallarooSuffix}", 
                    auth_endpoint=f"https://{wallarooPrefix}.keycloak.{wallarooSuffix}", 
                    auth_type="sso")

### Create Workspace

We will create a workspace to manage our pipeline and models.  The following variables will set the name of our sample workspace then set it as the current workspace for all other commands.

To allow this tutorial to be run multiple times or by multiple users in the same Wallaroo instance, a random 4 character prefix will be added to the workspace, pipeline, and model.

In [51]:
workspace_name = 'houseprice'
pipeline_name = 'housepriceabtest'
model_name_control = 'anomalyhousingcontrol'
model_file_name_control = './models/house_price_keras.onnx'
model_name_challenger = 'anomalyhousingchallenger'
model_file_name_challenger = './models/house_price_retrained.onnx'

In [40]:
def get_workspace(name):
    workspace = None
    for ws in wl.list_workspaces():
        if ws.name() == name:
            workspace= ws
    if(workspace == None):
        workspace = wl.create_workspace(name)
    return workspace

In [41]:
workspace = get_workspace(workspace_name)

wl.set_current_workspace(workspace)

{'name': 'houseprice', 'id': 152, 'archived': False, 'created_by': '138bd7e6-4dc8-4dc1-a760-c9e721ef3c37', 'created_at': '2023-03-13T20:43:33.923135+00:00', 'models': [{'name': 'anomalyhousingcontrol', 'versions': 2, 'owner_id': '""', 'last_update_time': datetime.datetime(2023, 3, 13, 20, 43, 50, 275376, tzinfo=tzutc()), 'created_at': datetime.datetime(2023, 3, 13, 20, 43, 36, 960968, tzinfo=tzutc())}], 'pipelines': [{'name': 'housepriceanomaly', 'create_time': datetime.datetime(2023, 3, 13, 20, 43, 52, 9192, tzinfo=tzutc()), 'definition': '[]'}]}

### Set Up the Champion and Challenger Models

We will be using two models.  The champion model was uploaded in the step `00_house-price-model-saga-anomaly-testing` as `anomalyhousingcontrol`, while the challenger model will be uploaded in the next steps.

1. `house_price_keras.onnx`: model trained to determine house prices.
2. `house_price_retrained.onnx`: a retrained version model trained to determine house prices.

### Upload the Challenger Model

We will upload the challenger model next labeled as `anomalyhousingchallenger`, and the control as `anomalyhousingcontrol`.

In [52]:
housing_model_control = wl.upload_model(model_name_control, model_file_name_control).configure()

In [43]:
housing_model_challenger = wl.upload_model(model_name_challenger, model_file_name_challenger).configure()

### Define The Pipeline

Here we will configure a pipeline with two models and set the control model with a random split chance of receiving 2/3 of the data.  Because this is a random split, it is possible for one model or the other to receive more inferences than a strict 2:1 ratio, but the more inferences are run, the more likely it is for the proper ratio split.

In [53]:
pipeline = (wl.build_pipeline(pipeline_name)
            .add_random_split([(2, housing_model_control), (1, housing_model_challenger )], "session_id"))

### Deploy the pipeline

Now we deploy the pipeline so we can run our inference through it.

In [54]:
experiment_pipeline = pipeline.deploy()

In [55]:
experiment_pipeline.status()

{'status': 'Running',
 'details': [],
 'engines': [{'ip': '10.48.1.80',
   'name': 'engine-6bc7cbc765-7khpb',
   'status': 'Running',
   'reason': None,
   'details': [],
   'pipeline_statuses': {'pipelines': [{'id': 'housepriceabtest',
      'status': 'Running'}]},
   'model_statuses': {'models': [{'name': 'anomalyhousingcontrol',
      'version': '30fd938b-90b8-42f7-92e7-b1058cddd4ac',
      'sha': '809c9f9a3016e5ab2190900d5fcfa476ee7411aa7a9ac5d4041d1cbe874cf8b9',
      'status': 'Running'}]}}],
 'engine_lbs': [{'ip': '10.48.1.79',
   'name': 'engine-lb-86bc6bd77b-mg5v9',
   'status': 'Running',
   'reason': None,
   'details': []}],
 'sidekicks': []}

# Run a single inference
Now we have our deployment set up let's run a single inference. In the results we will be able to see the inference results as well as which model the inference went to under model_id.  We'll run the inference request 5 times, with the odds are that the challenger model being run at least once.

In [62]:
results = []
if arrowEnabled is True:
    housing_input = pd.DataFrame.from_records({"tensor": [
                                                [
                                                    0.6752651953165153,
                                                    0.49993424710692347,
                                                    0.7386510547400537,
                                                    1.4527294113261855,
                                                    -0.08666382440547035,
                                                    -0.0713079330077084,
                                                    1.8870291307801872,
                                                    0.9294639723887077,
                                                    -0.305653139057544,
                                                    -0.6285378875598833,
                                                    0.29288456205300767,
                                                    1.181109967163617,
                                                    -0.65605032361317,
                                                    1.1203567680905366,
                                                    -0.20817781526102327,
                                                    0.9695503533113344,
                                                    2.823342771358126
                                                ]
                                            ]
                                        })
    # use dataframe JSON files
    for x in range(5):
        result = experiment_pipeline.infer(housing_input)
        # display(result)
        display(result.loc[:,["out._model_split", "out.dense_2"]])    
else:
    # use Wallaroo JSON files
    results.append(experiment_pipeline.infer({"tensor": [
                                                [
                                                    0.6752651953165153,
                                                    0.49993424710692347,
                                                    0.7386510547400537,
                                                    1.4527294113261855,
                                                    -0.08666382440547035,
                                                    -0.0713079330077084,
                                                    1.8870291307801872,
                                                    0.9294639723887077,
                                                    -0.305653139057544,
                                                    -0.6285378875598833,
                                                    0.29288456205300767,
                                                    1.181109967163617,
                                                    -0.65605032361317,
                                                    1.1203567680905366,
                                                    -0.20817781526102327,
                                                    0.9695503533113344,
                                                    2.823342771358126
                                                ]
                                            ]
                                        }))
    results.append(experiment_pipeline.infer({"tensor": [
                                                [
                                                    0.6752651953165153,
                                                    0.49993424710692347,
                                                    0.7386510547400537,
                                                    1.4527294113261855,
                                                    -0.08666382440547035,
                                                    -0.0713079330077084,
                                                    1.8870291307801872,
                                                    0.9294639723887077,
                                                    -0.305653139057544,
                                                    -0.6285378875598833,
                                                    0.29288456205300767,
                                                    1.181109967163617,
                                                    -0.65605032361317,
                                                    1.1203567680905366,
                                                    -0.20817781526102327,
                                                    0.9695503533113344,
                                                    2.823342771358126
                                                ]
                                            ]
                                        }))
    results.append(experiment_pipeline.infer({"tensor": [
                                                [
                                                    0.6752651953165153,
                                                    0.49993424710692347,
                                                    0.7386510547400537,
                                                    1.4527294113261855,
                                                    -0.08666382440547035,
                                                    -0.0713079330077084,
                                                    1.8870291307801872,
                                                    0.9294639723887077,
                                                    -0.305653139057544,
                                                    -0.6285378875598833,
                                                    0.29288456205300767,
                                                    1.181109967163617,
                                                    -0.65605032361317,
                                                    1.1203567680905366,
                                                    -0.20817781526102327,
                                                    0.9695503533113344,
                                                    2.823342771358126
                                                ]
                                            ]
                                        }))
    results.append(experiment_pipeline.infer({"tensor": [
                                                [
                                                    0.6752651953165153,
                                                    0.49993424710692347,
                                                    0.7386510547400537,
                                                    1.4527294113261855,
                                                    -0.08666382440547035,
                                                    -0.0713079330077084,
                                                    1.8870291307801872,
                                                    0.9294639723887077,
                                                    -0.305653139057544,
                                                    -0.6285378875598833,
                                                    0.29288456205300767,
                                                    1.181109967163617,
                                                    -0.65605032361317,
                                                    1.1203567680905366,
                                                    -0.20817781526102327,
                                                    0.9695503533113344,
                                                    2.823342771358126
                                                ]
                                            ]
                                        }))
    results.append(experiment_pipeline.infer({"tensor": [
                                                [
                                                    0.6752651953165153,
                                                    0.49993424710692347,
                                                    0.7386510547400537,
                                                    1.4527294113261855,
                                                    -0.08666382440547035,
                                                    -0.0713079330077084,
                                                    1.8870291307801872,
                                                    0.9294639723887077,
                                                    -0.305653139057544,
                                                    -0.6285378875598833,
                                                    0.29288456205300767,
                                                    1.181109967163617,
                                                    -0.65605032361317,
                                                    1.1203567680905366,
                                                    -0.20817781526102327,
                                                    0.9695503533113344,
                                                    2.823342771358126
                                                ]
                                            ]
                                        }))
    for result in results:
        print(result[0].model())
        print(result[0].data())

,out._model_split,out.dense_2
0,"[{""name"":""anomalyhousingcontrol"",""version"":""30fd938b-90b8-42f7-92e7-b1058cddd4ac"",""sha"":""809c9f9a3016e5ab2190900d5fcfa476ee7411aa7a9ac5d4041d1cbe874cf8b9""}]",[13.12781]


,out._model_split,out.dense_2
0,"[{""name"":""anomalyhousingchallenger"",""version"":""f636099a-00bd-481c-a258-a16a510cf645"",""sha"":""809c9f9a3016e5ab2190900d5fcfa476ee7411aa7a9ac5d4041d1cbe874cf8b9""}]",[13.12781]


,out._model_split,out.dense_2
0,"[{""name"":""anomalyhousingcontrol"",""version"":""30fd938b-90b8-42f7-92e7-b1058cddd4ac"",""sha"":""809c9f9a3016e5ab2190900d5fcfa476ee7411aa7a9ac5d4041d1cbe874cf8b9""}]",[13.12781]


,out._model_split,out.dense_2
0,"[{""name"":""anomalyhousingcontrol"",""version"":""30fd938b-90b8-42f7-92e7-b1058cddd4ac"",""sha"":""809c9f9a3016e5ab2190900d5fcfa476ee7411aa7a9ac5d4041d1cbe874cf8b9""}]",[13.12781]


,out._model_split,out.dense_2
0,"[{""name"":""anomalyhousingchallenger"",""version"":""f636099a-00bd-481c-a258-a16a510cf645"",""sha"":""809c9f9a3016e5ab2190900d5fcfa476ee7411aa7a9ac5d4041d1cbe874cf8b9""}]",[13.12781]


### Run Inference Batch

We will submit 1000 rows of test data through the pipeline, then loop through the responses and display which model each inference was performed in.  The results between the control and challenger should be approximately 2:1.

In [63]:
responses = []
if arrowEnabled is True:
    test_data = pd.read_json('./data/houseprice_inputs_500.json', orient="records")
    responses_anomaly = pd.DataFrame()
    # For each row, submit that row as a separate dataframe
    # Add the results to the responses array
    for index, row in test_data.head(500).iterrows():
        responses.append(experiment_pipeline.infer(row.to_frame('tensor').reset_index()))
        # display(responses)
    #now get our responses for each row
    # each r is a dataframe, then get the result from out.split into json and get the model name
    # for r in responses:
    #     display(r.loc[0]["out.split"])
    l = [json.loads(r.loc[0]["out._model_split"][0])["name"] for r in responses]
    df = pd.DataFrame({'model': l})
    display(df.model.value_counts())
else:
    l = []
    responses =[]
    from data import test_data
    for nth in range(500):
        responses.extend(experiment_pipeline.infer(test_data.data[nth]))
    l = [r.raw['model_name'] for r in responses]
    df = pd.DataFrame({'model': l})
    display(df.model.value_counts())

anomalyhousingcontrol       314
anomalyhousingchallenger    186
Name: model, dtype: int64

### Test Challenger

Now we have run a large amount of data we can compare the results.

For this experiment we are looking for a significant change in the fraction of inferences that predicted a probability of the seventh category being high than 0.5 so we can determine whether our challenger model is more "successful" than the champion model at identifying category 7.

In [65]:
control_count = 0
challenger_count = 0

control_success = 0
challenger_success = 0

if arrowEnabled is True:
    # do nothing
    for r in responses:
        if json.loads(r.loc[0]["out._model_split"][0])["name"] == "anomalyhousingcontrol":
            control_count += 1
            if(r.loc[0]["out.dense_2"][0] > .5):
                control_success += 1
        else:
            challenger_count += 1
            if(r.loc[0]["out.dense_2"][0] > .5):
               challenger_success += 1
else:
    for r in responses:
        if r.raw['model_name'] == "aloha-control":
            control_count += 1
            if(r.raw['outputs'][7]['Float']['data'][0] > .5):
                control_success += 1
        else:
            challenger_count +=1
            if(r.raw['outputs'][7]['Float']['data'][0] > .5):
                challenger_success += 1

print("control class 7 prediction rate: " + str(control_success/control_count))
print("challenger class 7 prediction rate: " + str(challenger_success/challenger_count))

control class 7 prediction rate: 1.0
challenger class 7 prediction rate: 1.0


### Logs

Logs can be viewed with the Pipeline method `logs()`.  For this example, only the first 5 logs will be shown.  For Arrow enabled environments, the model type can be found in the column `out._model_split`.

In [66]:
logs = experiment_pipeline.logs(limit=5)

if arrowEnabled is True:
    # display(logs)
    display(logs.loc[:,['time', 'out._model_split', 'out.dense_2']])
else:
    display(logs)

,time,out._model_split,out.dense_2
0,2023-03-13 22:43:01.982,"[{""name"":""anomalyhousingcontrol"",""version"":""30fd938b-90b8-42f7-92e7-b1058cddd4ac"",""sha"":""809c9f9a3016e5ab2190900d5fcfa476ee7411aa7a9ac5d4041d1cbe874cf8b9""}]",[12.946312]
1,2023-03-13 22:43:02.268,"[{""name"":""anomalyhousingcontrol"",""version"":""30fd938b-90b8-42f7-92e7-b1058cddd4ac"",""sha"":""809c9f9a3016e5ab2190900d5fcfa476ee7411aa7a9ac5d4041d1cbe874cf8b9""}]",[13.141247]
2,2023-03-13 22:43:02.567,"[{""name"":""anomalyhousingcontrol"",""version"":""30fd938b-90b8-42f7-92e7-b1058cddd4ac"",""sha"":""809c9f9a3016e5ab2190900d5fcfa476ee7411aa7a9ac5d4041d1cbe874cf8b9""}]",[12.689613]
3,2023-03-13 22:43:02.840,"[{""name"":""anomalyhousingcontrol"",""version"":""30fd938b-90b8-42f7-92e7-b1058cddd4ac"",""sha"":""809c9f9a3016e5ab2190900d5fcfa476ee7411aa7a9ac5d4041d1cbe874cf8b9""}]",[12.582939]
4,2023-03-13 22:43:03.119,"[{""name"":""anomalyhousingchallenger"",""version"":""f636099a-00bd-481c-a258-a16a510cf645"",""sha"":""809c9f9a3016e5ab2190900d5fcfa476ee7411aa7a9ac5d4041d1cbe874cf8b9""}]",[13.797941]


### Undeploy Pipeline

With the testing complete, we undeploy the pipeline to return the resources back to the environment.

In [67]:
experiment_pipeline.undeploy()